# Data visualization with various cuts explored in our paper.
## Transverse-momentum-dependent Multiplicities of Charged Hadrons for HERMES Airapetian:2012ki and COMPASS Collaborations Adolph:2013stb. 
## New COMPASS data are 
### Penn State Berks, Fall 2018 
#### * Alexey  Prokudin*

In [2]:
from __future__ import division, print_function
from functools import partial, reduce
from itertools import combinations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import fitpack
import interface
import obslib.sidis.reader
import obslib.sidis.stfuncs
from qcdlib.aux import AUX
from tools.config import conf
%matplotlib inline

This function reads data from multiple files and merges the data frames.

In [3]:
def get_data():
    # Merge all of the data tables
    return reduce(partial(pd.DataFrame.merge, how="outer"), map(pd.DataFrame, conf["sidis tabs"].values()))

upol_hermes_no_filters.py defines data files to read, no filters applied yet to the data selection. Here we read all HERMES multiplicities.

In [4]:
input_file = "../inputs_alexei_gk/upol_hermes_no_filters.py"

CUTMIN = 0.99
CUTMAX = 5.28

filterstr = "dy>"
#filterstr = "yh_minus_yp>"

The interface.py allows to get configuration file from the file that we read.

In [5]:
interface.gen_config(input_file)

In [6]:
interface.gen_config(input_file)  # Read file

conf["datasets"]["sidis"]["filters"][0] = filterstr + str(CUTMIN)  # Apply filter with min cut

conf["aux"] = AUX()
conf["sidis tabs"] = obslib.sidis.reader.READER().load_data_sets("sidis")
conf['sidis stfuncs'] = obslib.sidis.stfuncs.STFUNCS()

data = get_data().sort_values("dy")

loading sidis data sets 1000

KeyError: 'JAM3D'

In [ ]:
data["yh_minus_yp"] = data["yh"] - data["yp"] 
data["yi_minus_yp_over_yp"] = np.sqrt( ((data["yi"] - data["yp"])/data["yp"])**2 ) 
data["yf_minus_yh_over_yh"] = np.sqrt( ((data["yf"] - data["yh"])/data["yh"])**2 )
data["yi_minus_yp"] = np.sqrt( ((data["yi"] - data["yp"]))**2 ) 
data["yf_minus_yh"] = np.sqrt( ((data["yf"] - data["yh"]))**2 )

In [ ]:
print(", ".join(sorted(data.columns)))

We create QCUT which is qT^2/Q^2

In [ ]:
data["QCUT"] = (data["pT"] / data["z"])**2/ (data["Q2"])

In [ ]:

def custom_label(label):
    if label =="Q2":
        label="$Q^2$"
    if label =="qT":
        label="$q_T$"
    if label =="qT":
        label="$q_T$"
    if label =="W2":
        label="$W^2\; (GeV^2)$"
    if label =="qToverQ":
        label="$q_T/Q$"
    if label =="qToverQ2":
        label="$q_T^2/Q^2$"
    if label =="dy":
        label="$y_p-y_h$"
    if label =="yh_minus_yp":
        label="$y_h-y_p$"
    if label =="yi":
        label="$y_i$"
    if label =="yf":
        label="$y_f$"
    if label =="yh":
        label="$y_h$"
    if label =="yp":
        label="$y_p$"
    if label =="yi_minus_yp":
        label="$|y_i - y_p|$"
    if label =="yf_minus_yh":
        label="$|y_f - y_h|$"
    if label =="yi_minus_yp_over_yp":
        label="$|(y_i - y_p)/yp|$"
    if label =="yf_minus_yh_over_yh":
        label="$|(y_f - y_h)/yh|$"
    if label =="R":
        label="$|R|$"    
    if label =="lnR":
        label="$ln(|R|)$"    
    return label

def color_plot(data,vert_lab, hor_lab, cmap="plasma", alpha=1.0):
    vert = data[vert_lab].values
    hor = data[hor_lab].values

    data_q = data["QCUT"]

    colors = (data_q - min(data_q)) / (max(data_q) - min(data_q))  # XXX
    
    fig, ax = plt.subplots()

    #ax.set_title(custom_label(vert_lab) + " vs. " + custom_label(hor_lab))
    ax.set_ylabel(custom_label(vert_lab))
    ax.set_xlabel(custom_label(hor_lab))

    ax.scatter(hor, vert, c=colors, cmap=plt.get_cmap(cmap), alpha=alpha)
    
    #heatmap = ax.pcolor(data_q, cmap=plt.get_cmap(cmap)) # heatmap 
    #plt.colorbar(heatmap)

    fig.tight_layout()
    
    return fig, ax, vert, hor

In [ ]:
data["qT"] = data["pT"] / data["z"]
data["qToverQ"] = data["pT"] / data["z"]/(data["Q2"]**0.5)
data["qToverQ2"] = (data["pT"] / data["z"])**2/(data["Q2"])

In [ ]:
# labels = ("Q2", "pT", "x", "y", "z")
labels = ("qT","qToverQ2","Q2","dy","yh_minus_yp")

for v, h in combinations(labels, 2):
    fig, ax, vert, hor = color_plot(data,v, h, alpha=0.8)

    print(v + " vs. " + h)
    print("Number of points:", len(vert))
    print("Unique vertical values:", len(set(vert)))
    print("Unique horizontal values:", len(set(hor)))
    print()

# Let us study the whole HERMES data set

In [ ]:
len(data)

Kinematics of the data set

In [ ]:
print(np.min(data["x"]))
print(np.max(data["x"]))

print(np.min(data["z"]))
print(np.max(data["z"]))

print(np.min(data["Q2"]))
print(np.max(data["Q2"]))

print(np.min(data["Q2"]))
print(np.max(data["Q2"]))

print(np.min(data["pT"]))
print(np.max(data["pT"]))

In [ ]:
data.keys()

In [ ]:
import seaborn as sns
f, ax = plt.subplots(figsize=(10, 8))
filtered_data = data.drop([
      'Ebeam','obs','value','stat_u','systabs_u','systrel','systole','i','target','hadron','col'
], axis=1)
corr = filtered_data.corr()
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

In [ ]:
print(len(data))
print(len(data.query("(pT/z)**2>=Q2")))
fig, ax, vert, hor = color_plot(data,"qToverQ2", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
ax.set_yscale("log")
ax.set_ylim([0.0001,100])
plt.axhline(1,0,6)
plt.savefig("hermes_data_all.pdf")
plt.show()

In [ ]:
fig, ax, vert, hor = color_plot(data,"W2", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
#ax.set_yscale("log")
#ax.set_ylim([0.0001,100])
#plt.axhline(1,0,6)
plt.savefig("hermes_data_all_W2.pdf")
plt.show()

In [ ]:
fig, ax, vert, hor = color_plot(data,"qToverQ2", "R",cmap="seismic", alpha=0.8)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylim([0.0001,100])
#plt.axhline(1,0,6)
plt.savefig("hermes_data_all_qt_vs_R.pdf")
plt.show()

In [ ]:
fig, ax, vert, hor = color_plot(data,"Q2", "x",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
#ax.set_yscale("log")
#ax.set_ylim([0.0001,100])
#plt.axhline(1,0,6)
#plt.savefig("hermes_data_all.pdf")
plt.show()

In [ ]:
fig, ax, vert, hor = color_plot(data,"R", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
ax.set_yscale("log")
ax.set_ylim([0.01,10000])
plt.axhline(1.,0,6)
plt.savefig("hermes_data_all_R.pdf")
plt.show()

In [ ]:
filtered_data = data.query("x<0.12 and x>0.094")
fig, ax, vert, hor = color_plot(filtered_data,"R", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
ax.set_yscale("log")
ax.set_ylim([0.01,10000])
plt.axhline(1.,0,6)
#plt.savefig("hermes_data_all_R.pdf")
plt.show()

In [ ]:
filtered_data["z"]

In [ ]:
fig, ax, vert, hor = color_plot(data,"lnR", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
#ax.set_yscale("log")
#ax.set_ylim([0.0001,10000])
plt.axhline(0,0,6)
plt.savefig("hermes_data_all_lnR.pdf")
plt.show()

In [ ]:
fig, ax, vert, hor = color_plot(data,"yh", "yp",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
#ax.set_yscale("log")
#ax.set_ylim([0.0001,10000])
#plt.axhline(0,0,6)
plt.show()

In [ ]:
fig, ax, vert, hor = color_plot(data,"yi", "yp",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
#ax.set_yscale("log")
#ax.set_ylim([0.0001,10000])
#plt.axhline(0,0,6)
plt.show()

In [ ]:
fig, ax, vert, hor = color_plot(data,"yf", "yh",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
#ax.set_yscale("log")
#ax.set_ylim([0.0001,10000])
#plt.axhline(0,0,6)
plt.show()

In [ ]:
fig, ax, vert, hor = color_plot(data,"yi_minus_yp", "yp",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
#ax.set_yscale("log")
#ax.set_ylim([0.0001,10000])
#plt.axhline(0,0,6)
plt.savefig("hermes_data_yi_minus_yp.pdf")
plt.show()

In [ ]:
fig, ax, vert, hor = color_plot(data,"yf_minus_yh", "yh",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
#ax.set_yscale("log")
#ax.set_ylim([0.0001,10000])
#plt.axhline(0,0,6)
plt.savefig("hermes_data_yf_minus_yh.pdf")
plt.show()

# Here we create filtered data with standard Torino cuts

In [ ]:
torino_data = data.query("z<0.6 and Q2>1.69 and pT>0.2 and pT<0.9")

In [ ]:
normal_length = len(torino_data)

len(torino_data.query("(pT/z)**2>=Q2"))

In [ ]:
len(torino_data)

In [ ]:
fig, ax, vert, hor = color_plot(torino_data,"qToverQ2", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
ax.set_yscale("log")
ax.set_ylim([0.01,100])
plt.axhline(1.,0,6)
plt.savefig("hermes_data_torino.pdf")
plt.show()


Let us see what happens if we add rapidity cut to Torino selection

In [ ]:
torino_data_rap = data.query("z<0.6 and Q2>1.69 and pT>0.2 and pT<0.9 and dy>2.5")
len(torino_data_rap)

In [ ]:
len(torino_data_rap.query("(pT/z)**2>=Q2"))

In [ ]:
fig, ax, vert, hor = color_plot(torino_data_rap,"qToverQ2", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
ax.set_yscale("log")
ax.set_ylim([0.01,100])
plt.axhline(1.,0,6)
plt.savefig("hermes_data_torino_rapidity.pdf")
plt.show()

# Here we create filtered data with TMD inspired cuts

In [ ]:
cut_data_qt = data.query("z>0.2 and z<0.6 and Q2>1.69 and (pT/z)**2<=0.15*Q2")
#cut_data_qt = data.query("z>0.2 and z<0.6 and Q2>1.69 and dy>2.5")
print(len(cut_data_qt))
fig, ax, vert, hor = color_plot(cut_data_qt,"qToverQ2", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
ax.set_yscale("log")
ax.set_ylim([0.001,1])
plt.axhline(0.15,0,6)
plt.savefig("hermes_data_rapidity.pdf")
plt.show()


# Here we explore R cut

In [ ]:
print(np.min(data["lnR"]))
print(np.max(data["lnR"]))

In [ ]:
#cut_data_R = data.query("z>0.2 and z<0.6 and Q2>1.69 and (pT/z)**2<=0.15*Q2 and lnR<-1.5")
cut_data_R = data.query("z>0.2 and z<0.6 and Q2>1.69 and lnR<-0.5")

print(len(cut_data_R))
fig, ax, vert, hor = color_plot(cut_data_R,"qToverQ2", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
ax.set_yscale("log")
ax.set_ylim([0.001,1])
plt.axhline(0.15,0,6)
plt.savefig("hermes_data_R.pdf")
plt.show()

In [ ]:
cut_data_R = data.query("z>0.2 and z<0.6 and Q2>1.69 and lnR<-2.5")

print(len(cut_data_R))
fig, ax, vert, hor = color_plot(cut_data_R,"qToverQ2", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
ax.set_yscale("log")
ax.set_ylim([0.001,1])
plt.axhline(0.15,0,6)
plt.savefig("hermes_data_R25.pdf")
plt.show()

In [ ]:
#torino_data_x = data.query("z<0.6 and Q2>1.69 and pT>0.2 and pT<0.9 and x<0.1 and target=='proton' and hadron=='pi+'")
torino_data_x = data.query("x<0.096 and x>0.094 and target=='proton' and hadron=='pi+'")

fig, ax, vert, hor = color_plot(torino_data_x,"qToverQ2", "dy",cmap="seismic", alpha=0.8)
#ax.yscale('symlog')
#ax.set_yscale("log")
#ax.set_ylim([0.01,100])
#plt.axhline(0.15,0,6)
plt.show()